In [39]:
import json
import openai
import re
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import os
import random
import string

model = "gpt-4-1106-preview"


api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

functions = [
    {
        "name": "test_regex",
        "description": "Run a regex against a group of strings in the test bank. Return the fraction of strings matching regex.",
        "parameters": {
            "type": "object",
            "properties": {
                "regex": {
                    "type": "string",
                    "description": "The regex to test",
                },
                "string_array": {
                    "type": "string",
                    "enum": ["1", "2"],
                    "description": "The id of the array of strings to test against",
                },
            },
            "required": ["regex"],
        },
    },
]

def make_api_call(messages):
   response = client.chat.completions.create(model=model,
                                           messages=messages,
                                           functions=functions
                                           )
   return response.choices[0]

def make_random_string():
    length = random.randint(1, 30)
    random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
    return random_string

test_bank_size = 20

random_strings = [make_random_string() for _ in range(test_bank_size)]
should_contain_strings = [s for s in random_strings if 'i' in s]
non_contain_strings = [s for s in random_strings if 'i' not in s]

def test_regex(regex: str, string_array: str):
    strings_to_test = should_contain_strings if string_array == '1' else non_contain_strings
    return len([s for s in should_contain_strings if re.search(regex, s)]) / len(strings_to_test)

def execute_function_call(message):
    arguments = json.loads(message.function_call.arguments)
    if message.function_call.name == "test_regex":
        result = test_regex(**arguments)
    else:
        result = f"Error: function {message['function_call']['name']} does not exist"
    print(f"Handling {message}. Result was {result}")
    return str(result)


In [69]:
messages

[{'role': 'user',
  'content': 'We have two arrays of strings. We want you to develop a regex through trial and error that matches at least 80% of the strings in array 1 while not matching more than 10% of strings in the array 2.\n             \n             You can run the tests yourself. Remember to supply both the regex and whether you want to test on array 1 or 2.\n             \n             Once you have found this regex, we want you to tell us what it is. Does this make sense to you?\n             '},
 ChatCompletionMessage(content="Yes, this makes perfect sense. I will develop a regular expression (regex) and test it against the two arrays of strings, adjusting it as necessary to meet the required criteria:\n\n- Match at least 80% of strings in array 1.\n- Match no more than 10% of strings in array 2.\n\nI will start with an initial regex and iterate on it based on the results of the tests. Let's begin. I'll run a test with a generic regex to see how it performs on both arrays,

In [89]:
messages = [{'role': 'user', 
             'content': """We have two arrays of strings. We want you to develop a regex through trial and error that matches at least 80% of the strings in array 1 while not matching more than 10% of strings in the array 2.
             
             You can run the tests yourself. Remember to supply both the regex and whether you want to test on array 1 or 2.
             At each stage while testing the regex, write a couple sentences describing your thinking about what you are about to try.
             Once you have found this regex, we want you to tell us what it is. Does this make sense to you?
             """}]
             

while len(messages) < 50:
    chat_response = make_api_call(messages)
    if chat_response.finish_reason == 'function_call' or chat_response.message.function_call:
        messages.append(chat_response.message)
        tool_result = execute_function_call(chat_response.message)
        messages.append({'role': 'function',
                        'content': tool_result,
                        'name': chat_response.message.function_call.name
                        })
    else:
        messages.append({'role': 'user', 'content': 'Please continue.'})

Handling ChatCompletionMessage(content="Certainly, let's begin the search for an effective regex pattern.\n\nTo start with, without any information on the contents of the strings in both arrays, I'll create a basic regex that tries to capture common elements, such as words or numbers. From there, we can refine our regex based on the test results.\n\nI will try a simple regex that matches a string containing one or more alphanumeric characters. The regex for this would be `\\w+`.\n\nLet's first run this regex on array 1 to see how many strings it matches.", role='assistant', function_call=FunctionCall(arguments='{"regex":"\\\\w+","string_array":"1"}', name='test_regex'), tool_calls=None). Result was 1.0
Handling ChatCompletionMessage(content="The current regex `\\w+` matches 100% of the strings in array 1 which is a good start since we want to match at least 80%. Now, let's test this same regex on array 2 to ensure it doesn't match more than 10% of those strings.", role='assistant', fun

In [90]:
messages

[{'role': 'user',
  'content': 'We have two arrays of strings. We want you to develop a regex through trial and error that matches at least 80% of the strings in array 1 while not matching more than 10% of strings in the array 2.\n             \n             You can run the tests yourself. Remember to supply both the regex and whether you want to test on array 1 or 2.\n             At each stage while testing the regex, write a couple sentences describing your thinking about what you are about to try.\n             Once you have found this regex, we want you to tell us what it is. Does this make sense to you?\n             '},
 {'role': 'user', 'content': 'Please continue.'},
 ChatCompletionMessage(content="Certainly, let's begin the search for an effective regex pattern.\n\nTo start with, without any information on the contents of the strings in both arrays, I'll create a basic regex that tries to capture common elements, such as words or numbers. From there, we can refine our regex b